# APIM ❤️ AI Agents

## MCP-enabled A2A Protocol agents lab (initial release)
![flow](../../images/a2a-agent-2-agent.gif)

Playground to experiment with [A2A-enabled](https://www.microsoft.com/en-us/microsoft-cloud/blog/2025/05/07/empowering-multi-agent-apps-with-the-open-agent2agent-a2a-protocol/?msockid=3fc737ab34566ad7248a2255359d6b2c) agents with [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. 

This lab includes the following MCP servers deployed as part of shared infrastructure:
- Basic oncall service: provides a tool to get a list of random people currently on-call with their status and time zone.
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.

MCP-enabled agents are then deployed within ACA (Azure Container Apps) as A2A Protocol agents, one built with Semantic Kernel, and another with Autogen.

This lab demonstrates the art of the possible of creating hetrogenous multi-agentic system with agents created using multiple orchestrators, and then allowing a single unifying protocol to communicate accross the through APIM for Authn/Authz

### Prerequisites

- [Python 3.13 or later](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

**Important** This lab follows a different architecture from other lab.

Please deploy the shared infrastructure from this [Notebook](deploy-a2a-infra-assests.ipynb) before proceeding

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 10:08:48.448446 


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [2]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')
    
    a2a_weather_containerapp_resource_name = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppResourceName', 'A2A (Weather) Agent Container App Resource Name')
    a2a_oncall_containerapp_resource_name = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppResourceName', 'A2A (Oncall) Agent Container App Resource Name')

    a2a_weather_a2a_agent_ep = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppFQDN', 'A2A (Weather) Agent Endpoint')
    a2a_oncall_a2a_agent_ep = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppFQDN', 'A2A (Oncall) Agent Endpoint')


⚙️ Running: az deployment group show --name mcp-a2a-agents -g lab-mcp-a2a-agents 
❌ Failed to retrieve deployment: mcp-a2a-agents ⌚ 10:08:52.032961 :3s] ERROR: (ResourceGroupNotFound) Resource group 'lab-mcp-a2a-agents' could not be found.
Code: ResourceGroupNotFound
Message: Resource group 'lab-mcp-a2a-agents' could not be found.



### Building Customisable A2A MCP-enabled Agent images for deployment in ACA

#### Semantic-Kernel

In [3]:
build = build + 1

a2a_sk_server_image = "a2a-sk-server"
a2a_sk_server_src = "src/a2a_servers/a2a_sk_mcp_agent"

utils.run(f"az acr build --image {a2a_sk_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {a2a_sk_server_src}/Dockerfile {a2a_sk_server_src}/. --no-logs", 
          "Generic A2A SK Server with MCP image was successfully built", "Failed to build the Generic A2A SK Server with MCP image")

NameError: name 'container_registry_name' is not defined

#### Autogen

In [ ]:
a2a_ag_server_image = "a2a-ag-server"
a2a_ag_server_src = "src/a2a_servers/a2a_ag_mcp_agent"

utils.run(f"az acr build --image {a2a_ag_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {a2a_ag_server_src}/Dockerfile {a2a_ag_server_src}/. --no-logs", 
          "Generic A2A Autogen Server with MCP image was successfully built", "Failed to build the Generic A2A Autogen Server with MCP image")

### 🧪 Deploying A2A Agents in ACA

Use the following parameters to customise the deployment:
title, mcp_url

In [ ]:
title="Weather"
mcp_url = "/weather"

utils.run(f'az containerapp secret set -n  {a2a_weather_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', "Weather A2A Server secret updated", "Weather A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_weather_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{a2a_sk_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} A2A_URL={apim_resource_gateway_url}/weather-agent-a2a APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Weather A2A SK Server with MCP deployment succeeded", "Weather A2A SK Server with MCP deployment failed")

In [ ]:
utils.print_info(f'A2A URL for Weather Agent: {apim_resource_gateway_url}/weather-agent-a2a')

In [ ]:
title="Oncall"
mcp_url = "/oncall"

utils.run(f'az containerapp secret set -n  {a2a_oncall_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', f"{title} A2A Server secret updated", f"{title} A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_oncall_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{a2a_ag_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} A2A_URL={apim_resource_gateway_url}/oncall-agent-a2a APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Oncall A2A SK Server with MCP deployment succeeded", "Oncall A2A SK Server with MCP deployment failed")

In [ ]:
utils.print_info(f'A2A URL for Oncall Agent: {apim_resource_gateway_url}/oncall-agent-a2a')

### 🧪 A2A Multi-Agent using Agent Framework
Running Agent using Agent Framework

In [ ]:
import asyncio
import os

import httpx
from a2a.client import A2ACardResolver, A2AClient
from agent_framework.a2a import A2AAgent

import nest_asyncio
nest_asyncio.apply()

auth_headers = {
    "api-key": f"{apim_subscription_key}"
    }

async def main(agent, message):
    """Demonstrates connecting to and communicating with an A2A-compliant agent."""
    # Get A2A agent host from environment
    a2a_agent_host = f"{apim_resource_gateway_url}/{agent}"

    print(f"Connecting to A2A agent at: {a2a_agent_host}")

    # Initialize A2ACardResolver
    async with httpx.AsyncClient(timeout=60.0, headers=auth_headers) as http_client:
        resolver = A2ACardResolver(httpx_client=http_client, base_url=a2a_agent_host)

        # Get agent card
        agent_card = await resolver.get_agent_card(relative_card_path="/.well-known/agent.json")
        print(f"Found agent: {agent_card.name} - {agent_card.description}")

        # Disable streaming if needed
        ## agent_card.capabilities.streaming=False

        # Create A2A agent instance
        agent = A2AAgent(
            name=agent_card.name,
            description=agent_card.description,
            agent_card=agent_card,
            url=a2a_agent_host,
            http_client=http_client,
        )

        # Invoke the agent and output the result
        print("\nSending message to A2A agent...")
        response = await agent.run(message)

        # Print the response
        print("\nAgent Response:")
        for message in response.messages:
            print(message.text)
            print("\n---\n")


if __name__ == "__main__":
    asyncio.run(main("weather-agent-a2a", "What is the weather forecast for London this week?"))
    asyncio.run(main("oncall-agent-a2a", "What is the on-call schedule for this week in CET timezone?"))

### 🧪 Engage A2A Agent through CLI
Run the output from the following cell in terminal to engage with deployed A2A agents

In [ ]:
print(f'uv run ./src/a2a_client --agent "{apim_resource_gateway_url}/oncall-agent-a2a" --apikey "{apim_subscription_key}"')

print(f'uv run ./src/a2a_client --agent "{apim_resource_gateway_url}/weather-agent-a2a" --apikey "{apim_subscription_key}"')

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.